In [10]:
import numpy as np
import math
import matplotlib.pyplot as plt  
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pandas as pd


import sys
import xlwt
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
import datetime,pickle,os,glob

from tqdm import tqdm_notebook as tqdm

def transfromData(trainRaw, testRaw,windosSize):  ##Train ratial, train, test
    sc = MinMaxScaler(feature_range = (0, 1))
    #npRaw = np.array(rawData)
    #scaledData = sc.fit_transform(npRaw.reshape(-1,1))  
    #trainData = np.array(scaledData[:trainCount]).reshape(-1,1)##train 
    #testData = np.array(scaledData[-testCount:]).reshape(-1,1)
    
    npTrain = sc.fit_transform(np.array(trainRaw).reshape(-1,1))
    npTest = sc.fit_transform(np.array(testRaw).reshape(-1,1))
    

    X_train, y_train = splitXy(npTrain,windosSize)
    X_test, y_test = splitXy(npTest,windosSize)
    return sc, X_train, y_train, X_test, y_test

def splitXy(data,windosSize):
    windows = []
    for i in range(windosSize, data.shape[0]):
        windows.append(data[i-windosSize:i, 0])
    np.random.shuffle(windows)
    X = []
    y = []
    for i in range(len(windows)):
        X.append(windows[i][:6])
        y.append(windows[i][-(windosSize-6):])
    X, y = np.array(X), np.array(y)
    #X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    return X,y




def buildModel():
    dnn_1_input = Input(shape=(6,),name='dnn_1_input')
    dnn_1 = Dense(128,activation="tanh", name='dnn_1')(dnn_1_input)
    dnn_2 = Dense(64,activation="tanh", name='dnn_2')(dnn_1)
    dnn_3 = Dense(32,activation="tanh", name='dnn_3')(dnn_2)
    dnn_4 = Dense(1, activation='tanh', name='dnn_4')(dnn_3)

    model = Model(dnn_1_input,dnn_4)
    model.summary()
    model.compile(loss='mse',
                  optimizer='adam')
    return model
def getStationList():
    with open('pickles/stationList.pickle', 'rb') as handle:
        stationList = pickle.load(handle)
    os.chdir('excelFiles/LSTM')
    replaceDict = ['.xls','LSTMresult']
    for direct in glob.glob("*.xls"):
        fileName = direct                                               
        for w in replaceDict:
            fileName = fileName.replace(w,'')
        
        if fileName in stationList:
            stationList.remove(fileName)
            print(fileName)
    os.chdir('../..')
    return stationList

def writeExcelHead(sheet1,epochs,station):
    sheet1.write(0,1,station)
    raw = 1
    for e in range(epochs):
        sheet1.write(raw,0,e+1)
        raw+=1
def fetchData(station,windosSize):

    with open('pickles/'+station+'2017trainRaw.pickle', 'rb') as handle:
        trainRawData = pickle.load(handle)
    with open('pickles/'+station+'2017testRaw.pickle', 'rb') as handle:
        testRawData = pickle.load(handle)
        
    sc, X_train, y_train, X_test, y_test = transfromData(trainRawData,testRawData,windosSize)
    return sc, X_train, y_train, X_test, y_test
def trainLR(sc, X_train, y_train, X_test, y_test):
    
    reg = LinearRegression().fit(X_train, y_train)
    predicted_LR = sc.inverse_transform(reg.predict(X_test))
    ori_y_test = sc.inverse_transform(y_test)
    

    MSEs = []
    MAEs = []
    for idx in range(24):
        mse = mean_squared_error(predicted_LR[:,idx], ori_y_test[:,idx])
        mae = mean_absolute_error(predicted_LR[:,idx], ori_y_test[:,idx])
        MSEs.append(mse)
        MAEs.append(mae)

    return MSEs, MAEs

In [11]:


stationList = getStationList()
col=1
for station in tqdm(stationList):

    
    
    sc, X_train, y_train, X_test, y_test = fetchData(station,30)
    mseList, maeList = trainLR(sc, X_train, y_train, X_test, y_test)
    
  
    book = xlwt.Workbook(encoding="utf-8")
    sheet1 = book.add_sheet("Sheet1")
    writeExcelHead(sheet1,24,station)
    row = 1
    for mse in mseList:
        sheet1.write(row,1,mse)
        row+=1
    row = 1
    for mae in maeList:
        sheet1.write(row,2,mae)
        row+=1
    book.save("excelFiles/LR/LRresult-"+station+".xls")


In [4]:
k = [1,2,3,4,5,6,7,8,9]
k[-6:]

[4, 5, 6, 7, 8, 9]

In [10]:
k = [1,2,3,4,5]
k[2:]



[3, 4, 5]

In [ ]:
        X.append(windows[i][:6])
        y.append(windows[i][-(windosSize-6):])